# 📑 Tutorials for PyPOTS Clustering Models

## 📀 Preparing the **PhysioNet-2012** dataset for this tutorial

In [1]:
from pypots.data.generating import gene_physionet2012

# Load the PhysioNet-2012 dataset, disable artificially-missing values for evaluation
physionet2012_dataset = gene_physionet2012(artificially_missing=False)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())


2023-04-29 08:42:14 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Database)...
2023-04-29 08:42:14 [INFO]: Starting preprocessing physionet_2012...


Dataset physionet_2012 has already been downloaded. Processing directly...
Dataset physionet_2012 has already been cached. Loading from cache directly...
Loaded successfully!
dict_keys(['n_classes', 'n_steps', 'n_features', 'train_X', 'train_y', 'val_X', 'val_y', 'test_X', 'test_y', 'scaler'])


In [2]:
# Assemble the datasets for training, validating, and testing.
import numpy as np

# don't need validation set
dataset_for_training = {
    "X": np.concatenate([physionet2012_dataset['train_X'], physionet2012_dataset['val_X']], axis=0),
    "y": np.concatenate([physionet2012_dataset['train_y'], physionet2012_dataset['val_y']], axis=0),
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "y": physionet2012_dataset['test_y'],
}


## 🚀 An exmaple of **CRLI** for clustering

In [3]:
from pypots.clustering import CRLI

# initialize the model
crli = CRLI(
    n_steps=physionet2012_dataset["n_steps"],
    n_features=physionet2012_dataset["n_features"],
    n_clusters=physionet2012_dataset["n_classes"],
    n_generator_layers=2,
    rnn_hidden_size=256,
    epochs=10,  # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    patience=5,  # here we set patience=5 to early stop the training if the evaluting loss doesn't decrease for 5 epoches. You can leave it to defualt as None to disable early stopping.
    learning_rate=1e-3,
    # device='cpu', # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    saving_path="tutorial_results/clustering/crli",  # set the path for saving tensorboard files
)


2023-04-28 19:13:11 [INFO]: No given device, using default device: cpu
2023-04-28 19:13:11 [INFO]: saving_path is set as tutorial_results/clustering/crli, the trained model will be saved to tutorial_results/clustering/crli/20230428_T191311, the tensorboard file will be saved to tutorial_results/clustering/crli/20230428_T191311/tensorboard
2023-04-28 19:13:11 [INFO]: Model initialized successfully with the number of trainable parameters: 3136324


In [4]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
crli.fit(train_set=dataset_for_training)


2023-04-28 19:19:27 [INFO]: epoch 0: training loss_generator 3.9668, train loss_discriminator 0.3903
2023-04-28 19:25:25 [INFO]: epoch 1: training loss_generator 3.8344, train loss_discriminator 0.3697
2023-04-28 19:31:20 [INFO]: epoch 2: training loss_generator 3.7827, train loss_discriminator 0.3618
2023-04-28 19:37:30 [INFO]: epoch 3: training loss_generator 3.7704, train loss_discriminator 0.3438
2023-04-28 19:43:18 [INFO]: epoch 4: training loss_generator 3.8988, train loss_discriminator 0.3212
2023-04-28 19:49:06 [INFO]: epoch 5: training loss_generator 4.0449, train loss_discriminator 0.3032
2023-04-28 19:53:03 [INFO]: epoch 6: training loss_generator 63.4697, train loss_discriminator 0.2943
2023-04-28 19:56:35 [INFO]: epoch 7: training loss_generator 66.5428, train loss_discriminator 0.2810
2023-04-28 20:00:10 [INFO]: epoch 8: training loss_generator 66.5228, train loss_discriminator 0.2687
2023-04-28 20:00:10 [INFO]: Exceeded the training patience. Terminating the training pro

In [5]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
crli_prediction = crli.cluster(dataset_for_testing)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [6]:
from pypots.utils.metrics import cal_rand_index, cal_cluster_purity

# calculate mean absolute error on the ground truth (artificially-missing values)
RI = cal_rand_index(crli_prediction, dataset_for_testing["y"])
CP = cal_cluster_purity(crli_prediction, dataset_for_testing["y"])

print("Testing clustering metrics: \n"
      f'RI: {RI}, \n'
      f'CP: {CP},\n'
      )


Testing clustering metrics: 
RI: 0.5360923422835675, 
CP: 0.8544620517097581,



## 🚀 An exmaple of **VaDER** for clustering

In [3]:
from pypots.clustering import VaDER

# initialize the model
vader = VaDER(
    n_steps=physionet2012_dataset["n_steps"],
    n_features=physionet2012_dataset["n_features"],
    n_clusters=physionet2012_dataset["n_classes"],
    rnn_hidden_size=128,
    d_mu_stddev=2,
    pretrain_epochs=20,
    epochs=10,  # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    patience=5,  # here we set patience=5 to early stop the training if the evaluting loss doesn't decrease for 5 epoches. You can leave it to defualt as None to disable early stopping.
    learning_rate=5e-4,
    # device='cpu', # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    saving_path="tutorial_results/clustering/vader",  # set the path for saving tensorboard files
)


2023-04-29 08:42:53 [INFO]: No given device, using default device: cpu
2023-04-29 08:42:53 [INFO]: saving_path is set as tutorial_results/clustering/vader, the trained model will be saved to tutorial_results/clustering/vader/20230429_T084253, the tensorboard file will be saved to tutorial_results/clustering/vader/20230429_T084253/tensorboard
2023-04-29 08:42:53 [INFO]: Model initialized successfully with the number of trainable parameters: 293642


In [4]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
vader.fit(train_set=dataset_for_training)


2023-04-29 08:56:23 [INFO]: epoch 0: training loss 0.6338
2023-04-29 08:57:24 [INFO]: epoch 1: training loss 0.3448
2023-04-29 08:58:25 [INFO]: epoch 2: training loss 0.3530
2023-04-29 08:59:27 [INFO]: epoch 3: training loss 0.3438
2023-04-29 09:00:28 [INFO]: epoch 4: training loss 0.3452
2023-04-29 09:01:29 [INFO]: epoch 5: training loss 0.3597
2023-04-29 09:02:30 [INFO]: epoch 6: training loss 0.3517
2023-04-29 09:03:30 [INFO]: epoch 7: training loss 0.3318
2023-04-29 09:04:29 [INFO]: epoch 8: training loss 0.3248
2023-04-29 09:05:29 [INFO]: epoch 9: training loss 0.3248
2023-04-29 09:05:29 [INFO]: Finished training.
2023-04-29 09:05:29 [INFO]: Saved successfully to tutorial_results/clustering/vader/20230429_T084253/VaDER.pypots.


In [ ]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
vader_prediction = vader.cluster(dataset_for_testing)

In [6]:
from pypots.utils.metrics import cal_rand_index, cal_cluster_purity

# calculate mean absolute error on the ground truth (artificially-missing values)
RI = cal_rand_index(vader_prediction, dataset_for_testing["y"])
CP = cal_cluster_purity(vader_prediction, dataset_for_testing["y"])

print("Testing clustering metrics: \n"
      f'RI: {RI}, \n'
      f'CP: {CP},\n'
      )


Testing clustering metrics: 
RI: 0.766797390260205, 
CP: 0.8653044203502919,

